In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import pi
import os

file_path = 'school_data.csv'  
data = pd.read_csv(file_path)

# Ensure 'Player' column exists and handle case sensitivity
data.columns = data.columns.str.strip().str.lower()

output_dir = 'player_radar_charts'
os.makedirs(output_dir, exist_ok=True)

# Function to normalize data between 0 and 1
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

# Columns for each of the five matrices
categories = {
    'Attacking Contribution': ['goals', 'assists', 'shots_on_target', 'xa', 'progressive_runs_attempted', 'crosses', 'accurate_crosses'],
    'Defensive Capability': ['defensive_duels', 'slide_tackles', 'interceptions', 'recoveries', 'recoveries_in_own_half', 'loose_ball_duels'],
    'Passing & Playmaking': ['total_passes', 'forward_passes', 'passes_into_final_third', 'passes_into_box', 'shot_assists', 'second_assists'],
    'Duels & Physical Presence': ['total_duels', 'aerial_duels', 'offensive_duels', 'fouls_drawn', 'touches_inside_box'],
    'Discipline & Goalkeeping': ['fouls', 'yellow_cards', 'red_cards', 'gk_stat_saves', 'gk_stat_reflex_saves']
}

# Generate radar chart for each player in the dataset
def radar_chart(data):
    # Ensure 'player' column exists in lowercase
    if 'player_name' not in data.columns:
        print("The dataset does not contain a 'Player' column.")
        return

    sns.set(style="whitegrid")  # Set seaborn style for better aesthetics

    for player_name in data['player_name'].unique():
        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': 'polar'})

        # Preparing data for each category
        values = []
        baseline_values = []
        labels = []
        for category, metrics in categories.items():
            available_metrics = [metric for metric in metrics if metric in data.columns]
            category_values = normalize(data[available_metrics].sum(axis=1))
            baseline_value = category_values.mean()
            
            # Player specific values
            player_values = category_values[data['player_name'] == player_name].values[0]
            
            values.append(player_values * 10)  # Scale value to 0-10
            baseline_values.append(baseline_value * 10)  # Scale baseline to 0-10
            labels.append(category)

        # Ensure radar values are on the same scale (0 to 10)
        values = [min(10, max(0, v)) for v in values]  # Cap values at 10
        baseline_values = [min(10, max(0, v)) for v in baseline_values]  # Cap values at 10

        # Radar chart setup
        values += values[:1]  # Close the radar chart circle
        baseline_values += baseline_values[:1]  # Close the radar chart circle
        
        N = len(labels)
        angles = [n / float(N) * 2 * pi for n in range(N)]
        angles += angles[:1]

        ax.set_theta_offset(pi / 2)
        ax.set_theta_direction(-1)

        plt.xticks(angles[:-1], labels, fontsize=12, color='navy')
        ax.plot(angles, baseline_values, linewidth=2, linestyle='dotted', label='Baseline (Average)', color='grey')
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=player_name, color='blue')
        ax.fill(angles, values, color='blue', alpha=0.3)

        # Annotate each value on the radar chart
        for i, value in enumerate(values[:-1]):
            ax.text(angles[i], value + 0.5, f'{value:.1f}', horizontalalignment='center', size=10, color='darkred', weight='semibold')

        ax.set_ylim(0, 10)  # Set the same scale for all radars with max 10
        
        plt.title(f'{player_name} Performance Radar Chart', size=15, color='darkblue', weight='bold', pad=20)
        plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

        # Save the figure to the output directory
        output_path = os.path.join(output_dir, f'{player_name}.png')
        plt.savefig(output_path)
        plt.close()

radar_chart(data)

In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import pi
import os

file_path = 'school_data.csv'  
data = pd.read_csv(file_path)

# Ensure 'Player' column exists and handle case sensitivity
data.columns = data.columns.str.strip().str.lower()

# Function to normalize data between 0 and 1, handling division by zero
def normalize(series):
    min_val = series.min()
    max_val = series.max()
    if max_val == min_val:
        return series - series  # Return a series of zeros if all values are the same
    return (series - min_val) / (max_val - min_val)

# Normalize all numeric columns in the data
data_numeric = data.select_dtypes(include=[np.number])
data_normalized = data_numeric.apply(normalize)

# Replace original numeric columns with normalized values
data[data_numeric.columns] = data_normalized

# Columns for each of the five matrices
categories = {
    'Attacking Contribution': ['goals', 'assists', 'shots_on_target', 'xa', 'progressive_runs_attempted', 'crosses', 'accurate_crosses'],
    'Defensive Capability': ['defensive_duels', 'slide_tackles', 'interceptions', 'recoveries', 'recoveries_in_own_half', 'loose_ball_duels'],
    'Passing & Playmaking': ['total_passes', 'forward_passes', 'passes_into_final_third', 'passes_into_box', 'shot_assists', 'second_assists'],
    'Duels & Physical Presence': ['total_duels', 'aerial_duels', 'offensive_duels', 'fouls_drawn', 'touches_inside_box'],
    'Discipline & Goalkeeping': ['fouls', 'yellow_cards', 'red_cards', 'gk_stat_saves', 'gk_stat_reflex_saves']
}

# Global DataFrame to store all metrics for each player
df_metrics = pd.DataFrame()
df_metrics_normalized = pd.DataFrame()

# Create directory for saving radar charts
output_dir = 'radar_chart_per_minute'
os.makedirs(output_dir, exist_ok=True)

# Generate radar chart for all players and calculate baseline
def radar_chart(data):
    global df_metrics, df_metrics_normalized

    # Ensure 'player' column exists in lowercase
    if 'player_name' not in data.columns or 'minutes played' not in data.columns:
        print("The dataset does not contain the required 'player_name' or 'minutes played' column.")
        return

    sns.set(style="whitegrid")  # Set seaborn style for better aesthetics

    # Preparing data for each player and baseline
    all_players_values = []
    players_metrics = []

    for _, player_data in data.iterrows():
        player_name = player_data['player_name']
        minutes_played = player_data['minutes played']

        if minutes_played == 0:
            print(f"Player {player_name} has 0 minutes played. Skipping chart generation.")
            continue

        # Preparing data for each category
        values = []
        category_values = {'player_name': player_name}
        for category, metrics in categories.items():
            available_metrics = [metric for metric in metrics if metric in data.columns]
            # Calculate per minute contribution for each metric, average them
            per_minute_data = player_data[available_metrics].div(minutes_played)
            category_average = per_minute_data.mean() * 10  # Scale value to 0-10
            values.append(category_average)
            category_values[category] = category_average

        all_players_values.append(values)
        players_metrics.append(category_values)

    # Create a DataFrame to store the values of all five metrics for all players
    df_metrics = pd.DataFrame(players_metrics)

    # Normalize df_metrics between 0 and 1, then scale to 0-10
    df_metrics_normalized = df_metrics.drop(columns=['player_name']).apply(normalize) * 10
    df_metrics_normalized['player_name'] = df_metrics['player_name']
    print(df_metrics_normalized)

    # Calculate baseline (average of all players)
    baseline_values = df_metrics_normalized.drop(columns=['player_name']).mean().tolist()
    baseline_metrics = df_metrics_normalized.drop(columns=['player_name']).mean()
    print('Baseline Metrics for Each Category:')
    print(baseline_metrics)

    # Radar chart for each player including baseline
    labels = list(categories.keys())
    baseline_values = baseline_values + [baseline_values[0]]  # Close the radar chart circle to ensure five sides
    
    N = len(labels)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    for _, player_data in df_metrics_normalized.iterrows():
        player_name = player_data['player_name']
        player_values = player_data.drop('player_name').tolist()
        player_values += player_values[:1]  # Close the radar chart circle to ensure five sides

        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': 'polar'})

        ax.set_theta_offset(pi / 2)
        ax.set_theta_direction(-1)

        plt.xticks(angles[:-1], labels, fontsize=12, color='navy')
        ax.plot(angles, baseline_values, linewidth=2, linestyle='solid', label='Baseline (Average)', color='orange')
        ax.fill(angles, baseline_values, color='orange', alpha=0.3)

        ax.plot(angles, player_values, linewidth=2, linestyle='solid', label=player_name, color='blue')
        ax.fill(angles, player_values, color='blue', alpha=0.3)

        # Annotate each value on the radar chart
        for i, value in enumerate(player_values[:-1]):
            ax.text(angles[i], value + 0.5, f'{value:.1f}', horizontalalignment='center', size=10, color='darkred', weight='semibold')

        ax.set_ylim(0, 10)  # Set the same scale for all radars with max 10
        
        plt.title(f'{player_name} Performance Radar Chart', size=15, color='darkblue', weight='bold', pad=20)
        plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

        # Save the figure to the output directory
        output_path = os.path.join(output_dir, f'{player_name}.png')
        plt.savefig(output_path)
        plt.close()

radar_chart(data)

# Output the global dataframe df_metrics_normalized
print('Normalized Metrics DataFrame:')
print(df_metrics_normalized)


Player S. Vainstein has 0 minutes played. Skipping chart generation.
     Attacking Contribution  Defensive Capability  Passing & Playmaking  \
0                  0.142854              1.010679              2.817290   
1                  0.761650              1.573913              5.763188   
2                  1.291564              1.830536              3.993867   
3                  0.000000              0.148420              0.753746   
4                  0.952636              0.831199              1.542280   
..                      ...                   ...                   ...   
248                0.403540              1.353065              6.000459   
249                0.713529              1.154314              1.241746   
250                0.000000              0.165754              0.766579   
251                0.265662              2.244535              3.476929   
252                1.822784              1.502153              4.038472   

     Duels & Physical Presence

posx and posy should be finite values
posx and posy should be finite values


Normalized Metrics DataFrame:
     Attacking Contribution  Defensive Capability  Passing & Playmaking  \
0                  0.142854              1.010679              2.817290   
1                  0.761650              1.573913              5.763188   
2                  1.291564              1.830536              3.993867   
3                  0.000000              0.148420              0.753746   
4                  0.952636              0.831199              1.542280   
..                      ...                   ...                   ...   
248                0.403540              1.353065              6.000459   
249                0.713529              1.154314              1.241746   
250                0.000000              0.165754              0.766579   
251                0.265662              2.244535              3.476929   
252                1.822784              1.502153              4.038472   

     Duels & Physical Presence  Discipline & Goalkeeping       player

NameError: name 'df_metrics_normalized' is not defined

In [43]:
df_metrics_normalized.describe()

,Attacking Contribution,Defensive Capability,Passing & Playmaking,Duels & Physical Presence,Discipline & Goalkeeping
count,253.000000,253.000000,253.000000,253.000000,252.000000
mean,0.883787,1.662264,3.907919,1.774690,0.339882
std,0.877131,0.874755,1.921446,1.071459,0.633349
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.339331,1.299437,2.564528,1.140370,0.184381
50%,0.783706,1.641805,3.852931,1.625418,0.271367
75%,1.229958,2.002771,5.026643,2.300574,0.393139
max,10.000000,10.000000,10.000000,10.000000,10.000000


In [44]:
df_metrics_normalized[df_metrics_normalized['player_name'] == 'B. Keating']

,Attacking Contribution,Defensive Capability,Passing & Playmaking,Duels & Physical Presence,Discipline & Goalkeeping,player_name
22,10.0,10.0,10.0,10.0,10.0,B. Keating


In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import pi, log
import os

file_path = 'school_data.csv'  
data = pd.read_csv(file_path)

# Ensure 'Player' column exists and handle case sensitivity
data.columns = data.columns.str.strip().str.lower()

# Function to normalize data between 0 and 1, handling division by zero
def normalize(series):
    min_val = series.min()
    max_val = series.max()
    if max_val == min_val:
        return series - series  # Return a series of zeros if all values are the same
    return (series - min_val) / (max_val - min_val)

# Normalize all numeric columns in the data
data_numeric = data.select_dtypes(include=[np.number])
data_normalized = data_numeric.apply(normalize)

# Replace original numeric columns with normalized values
data[data_numeric.columns] = data_normalized

# Columns for each of the five matrices
categories = {
    'Attacking Contribution': ['goals', 'assists', 'shots_on_target', 'xa', 'progressive_runs_attempted', 'crosses', 'accurate_crosses'],
    'Defensive Capability': ['defensive_duels', 'slide_tackles', 'interceptions', 'recoveries', 'recoveries_in_own_half', 'loose_ball_duels'],
    'Passing & Playmaking': ['total_passes', 'forward_passes', 'passes_into_final_third', 'passes_into_box', 'shot_assists', 'second_assists'],
    'Duels & Physical Presence': ['total_duels', 'aerial_duels', 'offensive_duels', 'fouls_drawn', 'touches_inside_box'],
    'Discipline & Goalkeeping': ['fouls', 'yellow_cards', 'red_cards', 'gk_stat_saves', 'gk_stat_reflex_saves']
}

# Global DataFrame to store all metrics for each player
df_metrics = pd.DataFrame()
df_metrics_normalized = pd.DataFrame()

# Create directory for saving radar charts
output_dir = 'radar_chart_per_minute_log'
os.makedirs(output_dir, exist_ok=True)

# Generate radar chart for all players and calculate baseline
def radar_chart(data):
    global df_metrics, df_metrics_normalized

    # Ensure 'player' column exists in lowercase
    if 'player_name' not in data.columns or 'minutes played' not in data.columns:
        print("The dataset does not contain the required 'player_name' or 'minutes played' column.")
        return

    sns.set(style="whitegrid")  # Set seaborn style for better aesthetics

    # Preparing data for each player and baseline
    all_players_values = []
    players_metrics = []

    for _, player_data in data.iterrows():
        player_name = player_data['player_name']
        minutes_played = player_data['minutes played']

        if minutes_played == 0:
            print(f"Player {player_name} has 0 minutes played. Skipping chart generation.")
            continue

        # Preparing data for each category
        values = []
        category_values = {'player_name': player_name}
        for category, metrics in categories.items():
            available_metrics = [metric for metric in metrics if metric in data.columns]
            # Calculate contribution per log(minute_played + 1) for each metric, average them
            log_minutes_played = log(minutes_played + 1)
            per_log_minute_data = player_data[available_metrics].div(log_minutes_played)
            category_average = per_log_minute_data.mean() * 10  # Scale value to 0-10
            values.append(category_average)
            category_values[category] = category_average

        all_players_values.append(values)
        players_metrics.append(category_values)

    # Create a DataFrame to store the values of all five metrics for all players
    df_metrics = pd.DataFrame(players_metrics)

    # Normalize df_metrics between 0 and 1, then scale to 0-10
    df_metrics_normalized = df_metrics.drop(columns=['player_name']).apply(normalize) * 10
    df_metrics_normalized['player_name'] = df_metrics['player_name']
    print(df_metrics_normalized)

    # Calculate baseline (average of all players)
    baseline_values = df_metrics_normalized.drop(columns=['player_name']).mean().tolist()
    baseline_metrics = df_metrics_normalized.drop(columns=['player_name']).mean()
    print('Baseline Metrics for Each Category:')
    print(baseline_metrics)

    # Radar chart for each player including baseline
    labels = list(categories.keys())
    baseline_values = baseline_values + [baseline_values[0]]  # Close the radar chart circle to ensure five sides
    
    N = len(labels)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    for _, player_data in df_metrics_normalized.iterrows():
        player_name = player_data['player_name']
        player_values = player_data.drop('player_name').tolist()
        player_values += player_values[:1]  # Close the radar chart circle to ensure five sides

        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': 'polar'})

        ax.set_theta_offset(pi / 2)
        ax.set_theta_direction(-1)

        plt.xticks(angles[:-1], labels, fontsize=12, color='navy')
        ax.plot(angles, baseline_values, linewidth=2, linestyle='solid', label='Baseline (Average)', color='orange')
        ax.fill(angles, baseline_values, color='orange', alpha=0.3)

        ax.plot(angles, player_values, linewidth=2, linestyle='solid', label=player_name, color='blue')
        ax.fill(angles, player_values, color='blue', alpha=0.3)

        # Annotate each value on the radar chart
        for i, value in enumerate(player_values[:-1]):
            ax.text(angles[i], value + 0.5, f'{value:.1f}', horizontalalignment='center', size=10, color='darkred', weight='semibold')

        ax.set_ylim(0, 10)  # Set the same scale for all radars with max 10
        
        plt.title(f'{player_name} Performance Radar Chart', size=15, color='darkblue', weight='bold', pad=20)
        plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

        # Save the figure to the output directory
        output_path = os.path.join(output_dir, f'{player_name}.png')
        plt.savefig(output_path)
        plt.close()

radar_chart(data)

# Output the global dataframe df_metrics_normalized
print('Normalized Metrics DataFrame:')
print(df_metrics_normalized)


Player S. Vainstein has 0 minutes played. Skipping chart generation.
     Attacking Contribution  Defensive Capability  Passing & Playmaking  \
0                  0.193775              1.375082              3.460845   
1                  1.034112              2.144255              6.936791   
2                  1.487227              2.092892              4.029128   
3                  0.000000              0.182619              0.947496   
4                  1.203559              1.043013              1.786498   
..                      ...                   ...                   ...   
248                0.554800              1.868292              7.314449   
249                0.821580              1.311068              1.277970   
250                0.000000              0.236258              1.097181   
251                0.302865              2.544454              3.471862   
252                2.103782              1.717572              4.084449   

     Duels & Physical Presence

posx and posy should be finite values
posx and posy should be finite values


Normalized Metrics DataFrame:
     Attacking Contribution  Defensive Capability  Passing & Playmaking  \
0                  0.193775              1.375082              3.460845   
1                  1.034112              2.144255              6.936791   
2                  1.487227              2.092892              4.029128   
3                  0.000000              0.182619              0.947496   
4                  1.203559              1.043013              1.786498   
..                      ...                   ...                   ...   
248                0.554800              1.868292              7.314449   
249                0.821580              1.311068              1.277970   
250                0.000000              0.236258              1.097181   
251                0.302865              2.544454              3.471862   
252                2.103782              1.717572              4.084449   

     Duels & Physical Presence  Discipline & Goalkeeping       player

In [47]:

# normal distributed 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import pi
import scipy.stats as stats
import os

file_path = 'school_data.csv'  
data = pd.read_csv(file_path)

# Ensure 'Player' column exists and handle case sensitivity
data.columns = data.columns.str.strip().str.lower()

# Function to normalize data between 0 and 1, handling division by zero
def normalize(series):
    min_val = series.min()
    max_val = series.max()
    if max_val == min_val:
        return series - series  # Return a series of zeros if all values are the same
    return (series - min_val) / (max_val - min_val)

# Normalize all numeric columns in the data
data_numeric = data.select_dtypes(include=[np.number])
data_normalized = data_numeric.apply(normalize)

# Replace original numeric columns with normalized values
data[data_numeric.columns] = data_normalized

# Columns for each of the five matrices
categories = {
    'Attacking Contribution': ['goals', 'assists', 'shots_on_target', 'xa', 'progressive_runs_attempted', 'crosses', 'accurate_crosses'],
    'Defensive Capability': ['defensive_duels', 'slide_tackles', 'interceptions', 'recoveries', 'recoveries_in_own_half', 'loose_ball_duels'],
    'Passing & Playmaking': ['total_passes', 'forward_passes', 'passes_into_final_third', 'passes_into_box', 'shot_assists', 'second_assists'],
    'Duels & Physical Presence': ['total_duels', 'aerial_duels', 'offensive_duels', 'fouls_drawn', 'touches_inside_box'],
    'Discipline & Goalkeeping': ['fouls', 'yellow_cards', 'red_cards', 'gk_stat_saves', 'gk_stat_reflex_saves']
}

# Global DataFrame to store all metrics for each player
df_metrics = pd.DataFrame()
df_metrics_normalized = pd.DataFrame()

# Create directory for saving radar charts
output_dir = 'radar_chart_per_minute_normal_distribution'
os.makedirs(output_dir, exist_ok=True)

# Generate radar chart for all players and calculate baseline
def radar_chart(data):
    global df_metrics, df_metrics_normalized

    # Ensure 'player' column exists in lowercase
    if 'player_name' not in data.columns or 'minutes played' not in data.columns:
        print("The dataset does not contain the required 'player_name' or 'minutes played' column.")
        return

    sns.set(style="whitegrid")  # Set seaborn style for better aesthetics

    # Normal distribution parameters
    mean_minutes = 15
    std_dev_minutes = 11

    # Preparing data for each player and baseline
    all_players_values = []
    players_metrics = []

    for _, player_data in data.iterrows():
        player_name = player_data['player_name']
        minutes_played = player_data['minutes played']

        if minutes_played == 0:
            print(f"Player {player_name} has 0 minutes played. Skipping chart generation.")
            continue

        # Calculate weight using the normal distribution PDF
        weight = stats.norm.pdf(minutes_played, mean_minutes, std_dev_minutes)

        # Preparing data for each category
        values = []
        category_values = {'player_name': player_name}
        for category, metrics in categories.items():
            available_metrics = [metric for metric in metrics if metric in data.columns]
            # Calculate contribution per (minutes played * weight) for each metric, average them
            per_weighted_minute_data = player_data[available_metrics].div(weight)
            category_average = per_weighted_minute_data.mean() * 10  # Scale value to 0-10
            values.append(category_average)
            category_values[category] = category_average

        all_players_values.append(values)
        players_metrics.append(category_values)

    # Create a DataFrame to store the values of all five metrics for all players
    df_metrics = pd.DataFrame(players_metrics)

    # Normalize df_metrics between 0 and 1, then scale to 0-10
    df_metrics_normalized = df_metrics.drop(columns=['player_name']).apply(normalize) * 10
    df_metrics_normalized['player_name'] = df_metrics['player_name']
    print(df_metrics_normalized)

    # Calculate baseline (average of all players)
    baseline_values = df_metrics_normalized.drop(columns=['player_name']).mean().tolist()
    baseline_metrics = df_metrics_normalized.drop(columns=['player_name']).mean()
    print('Baseline Metrics for Each Category:')
    print(baseline_metrics)

    # Radar chart for each player including baseline
    labels = list(categories.keys())
    baseline_values = baseline_values + [baseline_values[0]]  # Close the radar chart circle to ensure five sides
    
    N = len(labels)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    for _, player_data in df_metrics_normalized.iterrows():
        player_name = player_data['player_name']
        player_values = player_data.drop('player_name').tolist()
        player_values += player_values[:1]  # Close the radar chart circle to ensure five sides

        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': 'polar'})

        ax.set_theta_offset(pi / 2)
        ax.set_theta_direction(-1)

        plt.xticks(angles[:-1], labels, fontsize=12, color='navy')
        ax.plot(angles, baseline_values, linewidth=2, linestyle='solid', label='Baseline (Average)', color='orange')
        ax.fill(angles, baseline_values, color='orange', alpha=0.3)

        ax.plot(angles, player_values, linewidth=2, linestyle='solid', label=player_name, color='blue')
        ax.fill(angles, player_values, color='blue', alpha=0.3)

        # Annotate each value on the radar chart
        for i, value in enumerate(player_values[:-1]):
            ax.text(angles[i], value + 0.5, f'{value:.1f}', horizontalalignment='center', size=10, color='darkred', weight='semibold')

        ax.set_ylim(0, 10)  # Set the same scale for all radars with max 10
        
        plt.title(f'{player_name} Performance Radar Chart', size=15, color='darkblue', weight='bold', pad=20)
        plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

        # Save the figure to the output directory
        output_path = os.path.join(output_dir, f'{player_name}.png')
        plt.savefig(output_path)
        plt.close()

radar_chart(data)

# Output the global dataframe df_metrics_normalized
print('Normalized Metrics DataFrame:')
print(df_metrics_normalized)


Player S. Vainstein has 0 minutes played. Skipping chart generation.
     Attacking Contribution  Defensive Capability  Passing & Playmaking  \
0                  0.710386              3.788770              4.106039   
1                  3.800461              5.902373              7.759495   
2                  2.848302              2.828738              2.407772   
3                  0.000000              0.403981              1.246742   
4                  3.547689              2.246845              1.874387   
..                      ...                   ...                   ...   
248                2.105349              5.327219              8.424169   
249                1.572955              1.713924              0.904818   
250                0.000000              0.766607              1.774792   
251                0.544442              3.236845              1.968191   
252                4.085959              2.331378              2.473790   

     Duels & Physical Presence

posx and posy should be finite values
posx and posy should be finite values


Normalized Metrics DataFrame:
     Attacking Contribution  Defensive Capability  Passing & Playmaking  \
0                  0.710386              3.788770              4.106039   
1                  3.800461              5.902373              7.759495   
2                  2.848302              2.828738              2.407772   
3                  0.000000              0.403981              1.246742   
4                  3.547689              2.246845              1.874387   
..                      ...                   ...                   ...   
248                2.105349              5.327219              8.424169   
249                1.572955              1.713924              0.904818   
250                0.000000              0.766607              1.774792   
251                0.544442              3.236845              1.968191   
252                4.085959              2.331378              2.473790   

     Duels & Physical Presence  Discipline & Goalkeeping       player

In [49]:
data.head()

,player_name,team,position,minutes played,total_actions,successful_actions,goals,assists,shots,shots_on_target,...,gk_stat_shots_against,gk_stat_saves,gk_stat_reflex_saves,gk_stat_box_exits,gk_stat_passes_to_gk,gk_stat_passes_to_gk_completed,gk_stat_goal_kicks_attempted,gk_stat_short_goal_kicks,gk_stat_long_goal_kicks,ps
0,A. Adalsteinsson,Rutgers Scarlet Knights,"CB, LCB",0.825413,0.590907,0.547890,0.000000,0.000000,0.176871,0.126623,...,0.000000,0.00000,0.000000,0.000000,0.404762,0.404762,0.0,0.00,0.0,0
1,A. Barger,Indiana Hoosiers,LB,0.828530,0.715355,0.556074,0.038012,0.105263,0.138471,0.093301,...,0.000000,0.00000,0.000000,0.000000,0.184211,0.184211,0.0,0.00,0.0,0
2,A. Bilow,Ohio State Buckeyes,RW,0.345717,0.266250,0.201796,0.153664,0.255319,0.302938,0.289168,...,0.000000,0.00000,0.000000,0.000000,0.035461,0.035461,0.0,0.00,0.0,0
3,A. Braman,Wisconsin Badgers,GK,0.654043,0.195268,0.211752,0.000000,0.000000,0.000000,0.000000,...,0.492181,0.55873,0.569912,0.277551,0.000000,0.000000,0.2,0.04,0.3,0
4,A. Camara,Penn State Nittany Lion,"AMF, CF",0.602052,0.364143,0.172568,0.611111,0.205128,0.480159,0.431818,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0


In [51]:
df_metrics_normalized['position'] = data['position']

In [52]:
df_metrics_normalized

,Attacking Contribution,Defensive Capability,Passing & Playmaking,Duels & Physical Presence,Discipline & Goalkeeping,player_name,position
0,0.710386,3.788770,4.106039,2.672197,5.520008,A. Adalsteinsson,"CB, LCB"
1,3.800461,5.902373,7.759495,5.687252,1.066126,A. Barger,LB
2,2.848302,2.828738,2.407772,2.972874,2.263926,A. Bilow,RW
3,0.000000,0.403981,1.246742,0.094876,4.797361,A. Braman,GK
4,3.547689,2.246845,1.874387,5.982322,3.163403,A. Camara,"AMF, CF"
...,...,...,...,...,...,...,...
248,2.105349,5.327219,8.424169,3.091760,3.421489,Y. Senden,CF
249,1.572955,1.713924,0.904818,3.953314,1.325122,Z. Babiak,CB
250,0.000000,0.766607,1.774792,0.440181,7.325749,Z. Kelly,AMF
251,0.544442,3.236845,1.968191,1.537206,2.575965,Z. Martens,GK


In [53]:
df_metrics_normalized.to_csv('metrics5.csv', index=False)